In [15]:
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
import numpy as np
import matplotlib
from math import pi, log2, ceil

from sharedFunctions import runNoisy, runIdeal, initializeQReg

In [2]:
def CCP(qc, theta, A, B, T):
    """
    Multiple controlled phase shift
    :param qc: Quantum Circuit
    :param theta: phase shift amount
    :param A: Control 1
    :param B: Control 2
    :param T: Target
    :return:
    """
    qc.cp(theta, B, T)
    qc.cx(A, B)
    qc.cp(-theta, B, T)
    qc.cx(A, B)
    qc.cp(theta, A, T)

In [3]:
def addMultRow(qc, reg_a, s, reg_b, reg_p, limit):
    """
    Add a quantum register reg_b if reg_aVal is 1, and store the result in reg_p.
    :param qc:       quantum circuit that is being operated on
    :param reg_a:    the smaller of the two inputs, the multiplicand
    :param s:        the current index of register a
    :param reg_b:    the larger of the two inputs, the multiplier
    :param reg_p:    the qregister holding the resultant product
    :param limit: the minimum size that the phase shift can be
    """
    for b in range(0, len(reg_b)):
        for j in range(0, len(reg_b) - b):
            lam = np.pi / (2 ** (j + 1))
            if abs((j + 1)) <= limit:
                CCP(qc, lam, reg_a[s], reg_b[b], reg_p[b + j + s])
        for i in range(0, len(reg_p) - len(reg_b) - s):
            lam = np.pi / (2 ** (b + 2 + i))
            if abs((b + 2 + i)) <= limit:
                CCP(qc, lam, reg_a[s], reg_b[len(reg_b) - b - 1], reg_p[len(reg_b) + i + s])

In [4]:
def invQFT(qc, reg, limit):
    """
    Performs the inverse quantum Fourier transform on a register reg.
    Apply repeated phase rotations with parameters being pi divided by
    decreasing powers of two, and then apply a Hadamard gate to the nth qubit
    of the register reg.
    """
    for n in range(0, len(reg)):
        for j in range(0, n):
            qc.cp(-1 * pi / float(2**(n - j)), reg[j], reg[n])
        qc.h(reg[n])

In [5]:
def QFT(qc, reg, limit):
    """
    Computes the quantum Fourier transform of reg, one qubit at
    a time.
    Apply one Hadamard gate to the nth qubit of the quantum register reg, and
    then apply repeated phase rotations with parameters being pi divided by
    increasing powers of two.
    """
    for i in range(0, len(reg)):
        n = len(reg) - 1 - i
        qc.h(reg[n])
        for j in range(0, n):
            qc.cp(pi / float(2**(j + 1)), reg[n - (j + 1)], reg[n])

In [6]:
def invAQFT(qc, reg, limit):
    """
    Performs the inverse quantum Fourier transform on a register reg.
    Apply repeated phase rotations with parameters being pi divided by
    decreasing powers of two, and then apply a Hadamard gate to the nth qubit
    of the register reg.
    """
    for n in range(0, len(reg)):
        for j in range(0, n):
            if abs((n - j)) <= limit:
                qc.cp(-1 * pi / float(2**(n - j)), reg[j], reg[n])
        qc.h(reg[n])

In [7]:
def AQFT(qc, reg, limit):
    """
    Computes the quantum Fourier transform of reg, one qubit at
    a time.
    Apply one Hadamard gate to the nth qubit of the quantum register reg, and
    then apply repeated phase rotations with parameters being pi divided by
    increasing powers of two.
    """
    for i in range(0, len(reg)):
        n = len(reg) - 1 - i
        qc.h(reg[n])
        for j in range(0, n):
            if abs((j + 1)) <= limit:
                qc.cp(pi / float(2**(j + 1)), reg[n - (j + 1)], reg[n])

In [21]:
def createAQAMCircuit(multiplier, multiplicand, limit, readable=False):
    """
    Multiply two numbers using a weighted array structure
    :param multiplier: A binary string of the multiplier
    :param multiplicand: A binary string of the multiplicand
    :param limit: the minimum size that the phase shift can be
    :param readable: Whether to include barriers between stages (will increase circuit depth)
    :return: a QC built using the two input numbers and their binary lengths
    """
    # Take two numbers as user input in binary form
    len1 = len(multiplicand)
    len2 = len(multiplier)

    if (len1 >= 1) & (len2 >= 1):
        qrMultiplicand = QuantumRegister(len1, name="Multiplicand")  # Multiplicand
        qrMultiplier = QuantumRegister(len2, name="Multiplier")  # Multiplier
        qProduct = QuantumRegister(len1 + len2, name="product")  # holds both the final multiplied result
        CarrySum = ClassicalRegister(len1 + len2)  # Classical register to hold the final measured values

        qc = QuantumCircuit(qrMultiplicand, qrMultiplier, qProduct, CarrySum, name="qc2")

        # Store bit strings in quantum registers
        initializeQReg(qc, qrMultiplicand, multiplicand)
        initializeQReg(qc, qrMultiplier, multiplier)

        if readable: qc.barrier(label="Initialized + Start QFT")

        # Compute the Fourier transform of accumulator
        AQFT(qc, qProduct, limit)

        for i in range(0, len(qrMultiplicand)):
            if readable: qc.barrier(label=("Start of Row " + str(i)))

            addMultRow(qc, qrMultiplicand, i, qrMultiplier, qProduct, limit)

        if readable: qc.barrier(label="Done Looping")

        # Compute the inverse Fourier transform of accumulator
        invAQFT(qc, qProduct, limit)

        qc.measure(qProduct, CarrySum)

        return qc

In [30]:
def squareAQAMMultDepth(num, limit):
    """
    Generates the circuit for a square multiplication and returns the resulting depth

    :param num: The number being used as both the multiplier and multiplicand
    :param limit: the minimum size that the phase shift can be
    :return: The depth of the generated circuit
    """
    qc = createAQAMCircuit(num, num, limit)
    depth = qc.decompose().decompose().decompose().depth()
    ideal = runIdeal(qc)
    noisy = runNoisy(qc)
    del qc
    return depth

In [31]:
def identityAQAMMultDepth(num, limit):
    """
    Generates the circuit for an identity multiplication and returns the resulting depth

    :param num: The number being used as the multiplicand
    :param limit: the minimum size that the phase shift can be
    :return: The depth of the generated circuit
    """
    qc = createAQAMCircuit(num, "1", limit)
    depth = qc.decompose().decompose().decompose().depth()
    ideal = runIdeal(qc)
    noisy = runNoisy(qc)
    del qc
    return depth

In [38]:
def getDepths():
    """
    Tests a hardcoded sample of input sizes both for identity multiplication and square multiplication. Uses a limit of Log_2(n) for inputs of size n for approximate QFT operations Prints results to the console.

    :return: N/A
    """
    testArray = {"1": "1", "2": "11", "3": "111", "4": "1111", "5": "11111", "6": "111111"
                 #, "7": "1111111",
                 #"8": "11111111", "9": "111111111", "10": "1111111111",
                 #"11": "11111111111", "12": "111111111111"} #, "13": "1111111111111", "14": "11111111111111",
                 # "15": "111111111111111", "16": "1111111111111111", "17": "11111111111111111",
                 # "18": "111111111111111111", "19": "1111111111111111111", "20": "11111111111111111111"
                 }
    print("____________________________________")
    for num in testArray:
        limit = ceil(log2(len(testArray[num]))+2)
        print("Depth for an input of size {}".format(num))
        print("Limit for this input size {}".format(limit))
        depth = identityAQAMMultDepth(testArray[num], limit)
        print("identity depth: {}".format(depth))
        # print("identity ideal: ", ideal)
        # print("identity noisy: ", noisy)
        print("_____________")
        depth = squareAQAMMultDepth(testArray[num], limit)
        print("square depth: {}".format(depth))
        # print("square ideal: ", ideal)
        # print("square noisy: ", noisy)
        print("____________________________________")

In [ ]:
getDepths()
# Test a sample input (3x3)
# sample = "11111"
# qc = createAQAMCircuit(sample, sample, ceil(log2(len(sample))+2), readable=True)
# #Draw Circuit
# #qc.draw(output="mpl", style="iqp")
# print(qc.decompose().decompose().decompose().depth())
# runNoisy(qc)

____________________________________
Depth for an input of size 1
Limit for this input size 2
Counts(ideal): {'01': 1024}
Counts(noise): {'01': 963, '00': 34, '11': 24, '10': 3}
identity depth: 35
_____________
Counts(ideal): {'01': 1024}
Counts(noise): {'01': 961, '00': 41, '11': 22}
square depth: 35
____________________________________
Depth for an input of size 2
Limit for this input size 3
Counts(ideal): {'011': 1024}
Counts(noise): {'011': 964, '000': 10, '010': 22, '001': 25, '101': 1, '111': 2}
identity depth: 77
_____________
Counts(ideal): {'0001': 46, '1001': 978}
Counts(noise): {'0001': 40, '1001': 942, '1000': 15, '0011': 7, '0101': 17, '0110': 1, '1101': 2}
square depth: 149
____________________________________
Depth for an input of size 3
Limit for this input size 4
Counts(ideal): {'0111': 1024}
Counts(noise): {'0111': 982, '1111': 3, '0110': 23, '0001': 8, '0101': 2, '0011': 1, '0000': 4, '1011': 1}
identity depth: 129
_____________
Counts(ideal): {'110001': 985, '010001

In [13]:
# print(log2(len(sample)))

2.321928094887362
